In [39]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from os import listdir
from scipy import stats as sts

%cd "/data/groups/ag_kircher/"

/fast/home/groups/ag_kircher


In [27]:
samples = ['predictions_BH01.csv',
           'predictions_IH01.csv',
           'predictions_IH02.csv', 
           'predictions_IC15.csv', 
           'predictions_IC17.csv', 
           'predictions_IC20.csv', 
           'predictions_IC35.csv', 
           'predictions_IC37.csv']

In [36]:
samples = ["predictions_NPH001.csv", "predictions_NPH002.csv", "predictions_NPH003.csv", "predictions_NPH004.csv", "predictions_NPH005.csv", "predictions_P147_1.csv", "predictions_P147_3.csv", "predictions_P148_1.csv", "predictions_P148_3.csv", "predictions_C2_6.csv", "predictions_C2_7.csv", "predictions_P40_1.csv", "predictions_P40_2.csv"]

In [37]:
file_list = {}
for file in (samples):
    file_list[file] = pd.read_csv("cfDNA-reanalysis_Graz/kristin/snakemake_rfmodel/results/predictions/37/" + file, index_col = 0)

# Gene organizer set

In [40]:
gene_associations = pd.read_excel("cfDNA-reanalysis_Graz/kristin/rf_model/GeneORGANizer-Confident-typical-v12.xls")

gene_disease_associations = pd.read_csv("cfDNA-reanalysis_Graz/kristin/rf_model/diseases-organs.v12-1.csv")

In [30]:
pd.set_option('display.max_columns', None)

In [42]:
gene_associations

,Symbol,Name,cerebellum,brain,head,uterus,ear,outer ear,middle ear,inner ear,peripheral nerve,peripheral nervous system,cranial nerve,pituitary gland,ovary,testicle,penis,vagina,vulva,skin,hair,prostate,breast,eye,lower limb,spinal cord,mouth,skull,spinal column,vertebrae,pelvis,rib,rib cage,lung,foot,tibia,fibula,femur,sternum,chest wall,clavicle,scapula,humerus,radius,ulna,neck,mandible,maxilla,jaw,ankle,knee,hip,wrist,elbow,shoulder,finger,toe,digit,hand,forearm,arm,shin,thigh,upper limb,face,eyelid,kidney,urethra,heart,anus,urinary bladder,blood vessel,pancreas,bronchus,white blood cell,blood,liver,biliary tract,gallbladder,olfactory bulb,ureter,red blood cell,coagulation system,aorta,heart valve,rectum,tooth,nose,intestine,large intestine,forehead,diaphragm,stomach,small intestine,chin,cheek,bone marrow,lip,pharynx,nail,meninges,cerebrospinal fluid,spleen,lymph node,sinus,abdominal wall,duodenum,esophagus,placenta,larynx,trachea,vocal cord,epiglottis,tongue,scalp,lymph vessel,lacrimal apparatus,adrenal gland,sweat gland,salivary gland,thyroid,hypothalamus,appendix,parathyroid,thymus,fallopian tube,vas deferens
0,AARS,16,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABAT,18,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ABCA1,19,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ABCA3,21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ABCB7,22,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [33]:
import pybiomart as pbm 

##get biomart dataset 
dataset = pbm.Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')

##get gene name for each gene id in de_genes
names = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'])

In [34]:
go_organ_list = {}
for organ in ['blood', 'liver', 'lung', 'breast', 'large intestine', 'prostate']:
    
    genes_organ = gene_associations[gene_associations[organ] == 1]['Symbol']
    genes_organ = pd.DataFrame(genes_organ)
    gene_IDs_organ = names.merge(genes_organ, left_on = 'Gene name', right_on = 'Symbol')
    gene_IDs_organ= gene_IDs_organ['Gene stable ID'].unique()
    go_organ_list[organ] = gene_IDs_organ

In [35]:
for i in go_organ_list:
    print(i + ": " + str(len(go_organ_list[i])))

blood: 1915
liver: 1025
lung: 1610
breast: 483
large intestine: 896
prostate: 356


# Graz gene set

In [2]:
graz_blood = pd.read_csv("cfDNA-reanalysis_Graz/kristin/genesets_graz/blood.txt", sep = "\t").ensembl_gene_id
graz_colon = pd.read_csv("cfDNA-reanalysis_Graz/kristin/genesets_graz/colon.txt", sep = "\t").ensembl_gene_id
graz_prostate =  pd.read_csv("cfDNA-reanalysis_Graz/kristin/genesets_graz/prostate.txt", sep = "\t").ensembl_gene_id

In [21]:
np.intersect1d(graz_blood.values, graz_prostate.values).shape

(16855,)

In [25]:
np.unique(graz_prostate).shape

(16855,)

In [19]:
set(graz_blood) <= set(graz_colon)

False

In [10]:
graz_list = {'blood' : np.array(graz_blood), 'colon': np.array(graz_colon), 'prostate': np.array(graz_prostate)}


In [11]:
for i in graz_list:
    print(i + ": " + str(len(graz_list[i])))

blood: 16992
colon: 16870
prostate: 16871


# Plots

## Gene organizer

In [38]:
for organ in ['blood', 'liver', 'lung', 'breast', 'large intestine', 'prostate']:
    
    file_list_filtered = file_list.copy()
    
    for file in file_list_filtered: 
        file_list_filtered[file] = file_list_filtered[file].filter(go_organ_list[organ], axis = 0).predicted
        
    crosstable = pd.DataFrame(index = file_list_filtered.keys(), columns = file_list_filtered.keys(), dtype="float")

    for file1 in file_list_filtered:
        for file2 in file_list_filtered: 
            pvalue = sts.ranksums(file_list_filtered[file1], file_list_filtered[file2])
            crosstable.loc[file1,file2] = pvalue[1]
            
    log_crosstable = -np.log(crosstable) 

    log_crosstable.index = [x.replace("predictions_","").replace(".csv","") for x in list(file_list_filtered.keys())]
    log_crosstable.columns = [x.replace("predictions_","").replace(".csv","") for x in list(file_list_filtered.keys())]
    #display(crosstable)

    plot = sns.heatmap(data = log_crosstable, cmap = 'Spectral', cbar_kws={'label': '-log(pvalue)'})
    plot = plt.yticks(rotation=0) 
    plt.title("Rank sum test- " + organ + " gene set")
    plt.tight_layout()
    plt.savefig("cfDNA-reanalysis_Graz/kristin/snakemake_diffexp/results/" + organ + "_heatmap_goGenes_grazSamples.pdf", dpi = 90)
    plt.close()
    
    

## Graz plots

In [30]:
for organ in ['blood', 'colon', 'prostate']:
    
    file_list_filtered = file_list.copy()
    
    for file in file_list_filtered: 
        file_list_filtered[file] = file_list_filtered[file].filter(graz_list[organ], axis = 0).predicted
        
    crosstable = pd.DataFrame(index = file_list_filtered.keys(), columns = file_list_filtered.keys(), dtype="float")

    for file1 in file_list_filtered:
        for file2 in file_list_filtered: 
            pvalue = sts.ranksums(file_list_filtered[file1], file_list_filtered[file2])
            crosstable.loc[file1,file2] = pvalue[1]
            
    log_crosstable = -np.log(crosstable) 

    log_crosstable.index = [x.replace("predictions_","").replace(".csv","") for x in list(file_list_filtered.keys())]
    log_crosstable.columns = [x.replace("predictions_","").replace(".csv","") for x in list(file_list_filtered.keys())]
    display(crosstable)

    plot = sns.heatmap(data = log_crosstable, cmap = 'Spectral', cbar_kws={'label': '-log(pvalue)'})
    plot = plt.yticks(rotation=0) 
    plt.title("Rank sum test- " + organ + " gene set")
    plt.tight_layout()
    plt.savefig("cfDNA-reanalysis_Graz/kristin/snakemake_diffexp/results/" + organ + "_heatmap_grazGenes_cellSamples.pdf", dpi = 90)
    plt.close()
    
    

,predictions_BH01.csv,predictions_IH01.csv,predictions_IH02.csv,predictions_IC15.csv,predictions_IC17.csv,predictions_IC20.csv,predictions_IC35.csv,predictions_IC37.csv
predictions_BH01.csv,1.000000e+00,4.749237e-01,3.289457e-01,0.000135,1.354375e-12,6.647513e-02,1.845124e-09,1.709988e-05
predictions_IH01.csv,4.749237e-01,1.000000e+00,9.549629e-02,0.001879,2.133846e-10,2.824312e-01,1.065266e-07,3.372884e-04
predictions_IH02.csv,3.289457e-01,9.549629e-02,1.000000e+00,0.000001,6.456984e-16,4.303114e-03,2.643078e-12,1.075022e-07
predictions_IC15.csv,1.345382e-04,1.879123e-03,1.357995e-06,1.000000,1.092682e-03,3.817936e-02,2.477469e-02,6.327388e-01
predictions_IC17.csv,1.354375e-12,2.133846e-10,6.456984e-16,0.001093,1.000000e+00,5.172288e-08,3.430884e-01,5.276277e-03
predictions_IC20.csv,6.647513e-02,2.824312e-01,4.303114e-03,0.038179,5.172288e-08,1.000000e+00,1.449523e-05,1.047111e-02
predictions_IC35.csv,1.845124e-09,1.065266e-07,2.643078e-12,0.024775,3.430884e-01,1.449523e-05,1.000000e+00,7.713856e-02
predictions_IC37.csv,1.709988e-05,3.372884e-04,1.075022e-07,0.632739,5.276277e-03,1.047111e-02,7.713856e-02,1.000000e+00


,predictions_BH01.csv,predictions_IH01.csv,predictions_IH02.csv,predictions_IC15.csv,predictions_IC17.csv,predictions_IC20.csv,predictions_IC35.csv,predictions_IC37.csv
predictions_BH01.csv,1.000000e+00,4.688068e-01,3.252120e-01,0.000135,1.292201e-12,6.443031e-02,1.866556e-09,1.686650e-05
predictions_IH01.csv,4.688068e-01,1.000000e+00,9.200188e-02,0.001954,2.194010e-10,2.803111e-01,1.141207e-07,3.468447e-04
predictions_IH02.csv,3.252120e-01,9.200188e-02,1.000000e+00,0.000001,5.742904e-16,4.020539e-03,2.549148e-12,1.014067e-07
predictions_IC15.csv,1.350506e-04,1.953782e-03,1.312810e-06,1.000000,1.063030e-03,3.964655e-02,2.485319e-02,6.295671e-01
predictions_IC17.csv,1.292201e-12,2.194010e-10,5.742904e-16,0.001063,1.000000e+00,5.416246e-08,3.392395e-01,5.221251e-03
predictions_IC20.csv,6.443031e-02,2.803111e-01,4.020539e-03,0.039647,5.416246e-08,1.000000e+00,1.558396e-05,1.081429e-02
predictions_IC35.csv,1.866556e-09,1.141207e-07,2.549148e-12,0.024853,3.392395e-01,1.558396e-05,1.000000e+00,7.804947e-02
predictions_IC37.csv,1.686650e-05,3.468447e-04,1.014067e-07,0.629567,5.221251e-03,1.081429e-02,7.804947e-02,1.000000e+00


,predictions_BH01.csv,predictions_IH01.csv,predictions_IH02.csv,predictions_IC15.csv,predictions_IC17.csv,predictions_IC20.csv,predictions_IC35.csv,predictions_IC37.csv
predictions_BH01.csv,1.000000e+00,4.756537e-01,3.275914e-01,0.000136,1.360165e-12,6.712766e-02,1.826681e-09,1.707906e-05
predictions_IH01.csv,4.756537e-01,1.000000e+00,9.524612e-02,0.001886,2.127444e-10,2.839671e-01,1.048548e-07,3.354352e-04
predictions_IH02.csv,3.275914e-01,9.524612e-02,1.000000e+00,0.000001,6.403991e-16,4.333862e-03,2.578859e-12,1.058955e-07
predictions_IC15.csv,1.355874e-04,1.885535e-03,1.354607e-06,1.000000,1.089217e-03,3.791921e-02,2.453628e-02,6.312003e-01
predictions_IC17.csv,1.360165e-12,2.127444e-10,6.403991e-16,0.001089,1.000000e+00,5.066828e-08,3.448010e-01,5.296632e-03
predictions_IC20.csv,6.712766e-02,2.839671e-01,4.333862e-03,0.037919,5.066828e-08,1.000000e+00,1.406498e-05,1.032346e-02
predictions_IC35.csv,1.826681e-09,1.048548e-07,2.578859e-12,0.024536,3.448010e-01,1.406498e-05,1.000000e+00,7.688021e-02
predictions_IC37.csv,1.707906e-05,3.354352e-04,1.058955e-07,0.631200,5.296632e-03,1.032346e-02,7.688021e-02,1.000000e+00
